In [64]:
import pandas as pd
import util
from util import *

In [65]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [66]:
subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))

nice_subset = [tier_translations[x] for x in subset]
nice_subset

['Analysis and Modeling',
 'Analysis and Modeling: 3D Modeling',
 'Anatomical Target',
 'Anatomical Target: Lower Extremity',
 'Anatomical Target: Lower Extremity - Hip',
 'Anatomical Target: Lower Extremity - Knee',
 'Anatomical Target: Torso',
 'Anatomical Target: Torso - Spine',
 'Anatomical Target: Upper Extremity',
 'Anatomical Target: Upper Extremity - Shoulder',
 'Imaging',
 'Imaging: CT',
 'Imaging: MRI',
 'Imaging: Ultrasound',
 'Manufacturing',
 'Manufacturing: Additive Manufacturing',
 'Personalized Product: Guide or Jig',
 'Personalized Product: Implant',
 'Specification of Use',
 'Specification of Use: Disease',
 'Specification of Use: Joint Replacement',
 'Surgical Method']

In [80]:
import funcy as f
from random import shuffle

num_neg = 2

@f.collecting
def create_examples(row):
    abstract = row.abstract
    claims = row.claims
    yield (abstract, claims, True)
    for text in [abstract,claims]:
        neg_count = 0
        shuffle(subset)
        for tag in subset:
            if row[tag]:
                yield (text, f"Tagged as {tier_translations[tag]}.", True)
            elif neg_count < num_neg or all_neg:
                neg_count = neg_count + 1
                yield (text, f"Tagged as {tier_translations[tag]}.", False)
            else:
                continue

In [73]:
triplets = pd.DataFrame(training_set.apply(create_examples, axis=1).explode().tolist())

In [74]:
triplets.columns = ["sentence1", "sentence2", "label"]

In [75]:
triplets = triplets.drop_duplicates()

In [76]:
triplets.shape

(15797, 3)

In [77]:
triplets.to_parquet("triples.parquet")

In [78]:
testing_triplets = pd.DataFrame(testing_set.apply(create_examples, axis=1).explode().tolist())

In [79]:
testing_triplets.columns = ["sentence1", "sentence2", "label"]
testing_triplets = testing_triplets.drop_duplicates()
testing_triplets.shape

(4099, 3)

In [63]:
testing_triplets.to_parquet("testing_triples.parquet")

In [42]:
# from tqdm.auto import tqdm
# def build_data_set(data_set): 
#     frames = []
#     for tier in [tier1, tier2, tier3, tier4]:
#         for label in tqdm(set(tier) & set(all_tiers_100)):
#             nice_label = tier_translations.get(label)
#             if nice_label is None:
#                 continue
#             positives = data_set[data_set[label]].abstract.to_frame("positive")
#             positives['fk'] = 0
#             positives['label'] = f"This is labelled: {nice_label}."
#             negatives = data_set[~data_set[label]].abstract.to_frame("negative")
#             negatives['fk'] = 0
#             triplets = pd.merge(positives, negatives, how="outer", on="fk")
#             frames.append(triplets[['label', 'positive', 'negative']])
#     return pd.concat(frames)